In [1]:
'''
Important paths
'''
path_to_original_pascal3dp = '/srv/PASCAL3D+_release1.1/'
# bg_img_path = '/srv/PASCAL3D+_release1.1/Images/bus_imagenet/'
# bg_mask_path = '/srv/obj_mask/bus'
# bg_anno_path = '/srv/PASCAL3D+_release1.1/Annotations/bus_imagenet/'

occluder_path = '/srv/occluder_libs_test_small.npz'

bg_list_path = path_to_original_pascal3dp + 'Image_sets/%s_imagenet_val.txt'
bg_img_path = path_to_original_pascal3dp + 'Images/%s_imagenet'
bg_anno_path = path_to_original_pascal3dp + 'Annotations/%s_imagenet'
bg_mask_path = path_to_original_pascal3dp + 'obj_mask/%s'

In [2]:
'''
change occlusion percentage to use bounding boxes
'''

'\nchange occlusion percentage to use bounding boxes\n'

In [3]:
# '''
# Get a bounding box for the occluder
# Takes the path to a .mat annotation file
# '''

def get_properties(path):
    data = scipy.io.loadmat(path)
    width = data['record']['size'][0][0][0][0][0]
    height = data['record']['size'][0][0][0][0][1]
    bbox = data['record'][0, 0]['objects'][0, 0]['bbox'][0].astype(int)
    return width[0, 0], height[0, 0], bbox

In [4]:
import scipy

def load_one_annotation(anno_path):
    a = scipy.io.loadmat(anno_path)
    # I added the astype int here....
    bbox_ = a['record'][0][0][1][0][0][1][0].astype(int)
    w = a['record']['size'][0][0][0][0][0][0][0]
    h = a['record']['size'][0][0][0][0][1][0][0]
    num_obj = len(a['record'][0][0][1][0])
    return w, h, bbox_, num_obj != 1

In [5]:
load_one_annotation('/srv/PASCAL3D+_release1.1/Annotations/bus_imagenet/n02924116_51203.mat')

(500, 333, array([ 76,  28, 371, 307]), False)

In [6]:
# THIS NEEDS TO BE CHANGED TO SEGMENTATION MASK PERCENTAGES

'''
get overlap percentage
'''
def overlap_ratio(occluder_bb, occludee_bb):

    #top left and bottom right points
    occluder_x1, occluder_y1, occluder_x2, occluder_y2 = occluder_bb
    occludee_x1, occludee_y1, occludee_x2, occludee_y2 = occludee_bb

    # area of the foreground object
    occludee_area = (occludee_x2 - occludee_x1) * (occludee_y2 - occludee_y1)

    # area of the background object being covered by the foreground object
    overlap_area = max(0, min(occludee_x2, occluder_x2) - max(occludee_x1, occluder_x1)) * max(0, min(occludee_y2, occluder_y2) - max(occludee_y1, occluder_y1))

    # overlap over the total background object area
    return overlap_area / occludee_area

In [7]:
'''
overlap percentage using segmentation masks instead of bounding boxes
mask_map is the results of mix_masks!
'''

def check_occ_ratio_seg(mask_map, mask_obj):
    mask_obj = mask_obj > 10
    mask_map = mask_map > 0.5
    in_box_size = np.sum(mask_obj)
    in_box_value = np.sum(np.logical_and(mask_obj, mask_map))

    # out_box_size = np.sum(np.logical_not(mask_obj))
    # out_box_value = np.sum(np.logical_and(np.logical_not(mask_obj), mask_map))

    return in_box_value / in_box_size #, out_box_value / out_box_size

In [8]:
'''
Place the masks into their appropriate location  in the background

Tne returned mask will have 0s or 1s as values

THis is not done yet....
'''
def mix_mask(mask, bbox, bg_w, bg_h):

    background_mask = np.zeros((bg_h, bg_w), dtype=mask.dtype)
    
    # Extract bbox coordinates
    x1, y1, x2, y2 = bbox
    
    # Ensure the mask matches the bbox dimensions
    # if mask.shape[0] != h or mask.shape[1] != w:
    #     mask = cv2.resize(mask, (w, h))
    
    # Calculate corresponding mask coordinates
    mask_x1 = x1 - x
    mask_y1 = y1 - y
    mask_x2 = mask_x1 + (x2 - x1)
    mask_y2 = mask_y1 + (y2 - y1)
    
    # Place the mask in the background
    background_mask[y1:y2, x1:x2] = mask[mask_y1:mask_y2, mask_x1:mask_x2]
    
    return background_mask

In [9]:
import random

'''
Get a list of randomly chosen bounding boxes to occlude the background object above some threshold
This can be improved if we know the foreground image has to be some base scale to allow for above threshold occlusion
'''
def get_bbox_list(bg_bbox, bg_mask, fg_mask, bg_w, bg_h, fg_w, fg_h):

    bboxes = []

    occludee_x1 = bg_bbox[0]
    occludee_y1 = bg_bbox[1] 
    occludee_x2 = bg_bbox[2]
    occludee_y2 = bg_bbox[3]

    num_boxes = 20 # iterate over multiple boxes
    overlap_threshold = .1 # ensure a minimum amount of overlap

    # print(bg_bbox, fg_w, fg_h)
    
    for _ in range(num_boxes):
        # top left corner of the occluder bounding box:

        occluder_x1 = random.randint(max(0, occludee_x1 - fg_w), occludee_x2) # overlapping in the x-direction
        occluder_y1 = random.randint(max(0, occludee_y1 - fg_h), occludee_y2) # overlapping in the y-direction

        # TODO: Fix so doesnt exceed background image
        
        occluder_x2 = occluder_x1 + fg_w
        occluder_y2 = occluder_y1 + fg_h

        # Maybe this will fix sizing error??? (IT DOES)
        if occluder_x2 > bg_w or occluder_y2 > bg_h:
            continue

        occluder_bb = [occluder_x1, occluder_y1, occluder_x2, occluder_y2]
    
        #mask_map = mix_mask(fg_mask, occluder_bb, bg_w, bg_h)

        occluded_ratio = overlap_ratio(occluder_bb, bg_bbox)
        #occluded_ratio = check_occ_ratio_seg(mask_map, bg_mask)

        if occluded_ratio >= overlap_threshold:
            bboxes.append([occluder_x1, occluder_y1, occluder_x2, occluder_y2])
    
    return bboxes

In [10]:
'''
returns the score, composite image, and compositive mask. 
num scales is the number of different foreground scales to try.
'''
from libcom import OPAScoreModel

def get_optimal_location(fg_img, fg_mask, bg_img, bg_mask, bg_w, bg_h, bg_bbox, num_scales):

    net = OPAScoreModel(device=0, model_type='SimOPA')
    cache_dir = './cache'

    # from libcom.fopa_heat_map.source.prepare_multi_fg_scales import prepare_multi_fg_scales
    scaled_fg_dir, scaled_mask_dir, csv_path = prepare_multi_fg_scales(cache_dir, fg_img, fg_mask, bg_img, 16)

    score = 0
    optimal_bbox = None
    best_fg = None
    best_mask = None
    best_comp = None 
    best_comp_mask = None
    
    # iterate over the different foreground scales
    with open(csv_path, mode='r', newline='') as csv_file:
        csv_reader = csv.DictReader(csv_file)
        for row in csv_reader:
            fg_name   = '{}_{}_{}_{}.jpg'.format(row["fg_name"].split(".")[0],row["bg_name"].split(".")[0],int(row["newWidth"]),int(row["newHeight"]))
            mask_name = '{}_{}_{}_{}.jpg'.format(row["fg_name"].split(".")[0],row["bg_name"].split(".")[0],int(row["newWidth"]),int(row["newHeight"]))
            scale     = row['scale']
            fg_w = int(row['newWidth'])
            fg_h = int(row['newHeight'])
            
            save_name = fg_name.split(".")[0] + '_' + str(scale) + '.jpg'
            bg_img    = read_image_pil(bg_img)
            # these lines might not be necessary
            # fg_img    = read_image_pil(os.path.join(scaled_fg_dir, fg_name))
            fg_mask   = np.array(Image.open(os.path.join(scaled_mask_dir, mask_name)))
            bbox_list = get_bbox_list(bg_bbox, bg_mask, fg_mask, bg_w, bg_h, fg_w, fg_h)

            for bbox in bbox_list:
                comp, comp_mask = get_composite_image(os.path.join(scaled_fg_dir, fg_name), os.path.join(scaled_mask_dir, mask_name), bg_img, bbox)
                bbox_score = net(comp, comp_mask)
                if bbox_score > score:
                    best_fg = os.path.join(scaled_fg_dir, fg_name)
                    best_mask = os.path.join(scaled_mask_dir, mask_name)
                    optimal_bbox = bbox
                    best_comp = comp
                    best_comp_mask = comp_mask
                    score = bbox_score

        return score, best_fg, best_mask, optimal_bbox, best_comp, best_comp_mask

No module 'xformers'. Proceeding without it.


/opt/conda/envs/Libcom/lib/python3.8/site-packages/pytorch_lightning/utilities/distributed.py:258: LightningDeprecationWarning: `pytorch_lightning.utilities.distributed.rank_zero_only` has been deprecated in v1.8.1 and will be removed in v2.0.0. You can import it from `pytorch_lightning.utilities` instead.
  rank_zero_deprecation(


In [11]:
import random
import numpy as np
from PIL import Image
import cv2

'''
Get a random occluder and correspondiong mask
'''
def get_occluder():
    data = np.load(occluder_path, allow_pickle=True)
    
    i = random.randint(0, len(data['images']))
    occ_img = f'./occluders/fg_img_{i}.jpg'
    occ_mask = f'./occluders/fg_mask_{i}.png'

    image = data['images'][i]
    mask = data['masks'][i]
    if mask.max() <= 1.0:  # Check if values are between 0 and 1
        mask = (mask * 255).astype(np.uint8)  # Scale to 0-255 and convert to uint8
    elif mask.dtype != np.uint8:  # If it's already 0-255 but not uint8
        mask = mask.astype(np.uint8)
        
    cv2.imwrite(occ_img, image)
    cv2.imwrite(occ_mask, mask)

    return occ_img, occ_mask

In [29]:
get_occluder()

(126, 64)
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


('./occluders/fg_img_1879.jpg', './occluders/fg_mask_1879.png')

In [12]:
from libcom import color_transfer
from libcom.utils.process_image import *
from libcom.utils.environment import *
from libcom import OPAScoreModel
from libcom import get_composite_image
from libcom.utils.process_image import make_image_grid
import cv2
import csv
from PIL import Image
from libcom.fopa_heat_map.source.prepare_multi_fg_scales import prepare_multi_fg_scales
from libcom import Mure_ObjectStitchModel
from libcom import ControlComModel

def generate_composite_image(bg_img, bg_w, bg_h, bg_mask, bg_bbox):

    net = Mure_ObjectStitchModel(device=0, sampler='plms')

    # fg_mask is a path to an image here.... but bg_mask is a numpy array!
    fg_img, fg_mask = get_occluder()
    
    score, fg_img, fg_mask, bbox, comp, comp_mask = get_optimal_location(fg_img, fg_mask, bg_img, bg_mask, bg_w, bg_h, bg_bbox, num_scales=16)
    
    if(score):
        res, show_fg_img = net(bg_img, [fg_img], [fg_mask], bbox, sample_steps=25, num_samples=3)
        bg_img   = draw_bbox_on_image(bg_img, bbox)
        grid_img = make_image_grid([bg_img, fg_img, comp, comp_mask] + [res[i] for i in range(3)])
        cv2.imwrite(f'./results/small_comp_{score}.jpg', grid_img)

In [13]:
#import BboxTools as bbt

def generate_dataset(cate, file_list, bg_img_dir, bg_anno_dir, bg_mask_dir, record_file):
    for file_name in file_list[:1]:
        bg_w, bg_h, bg_bbox, flag_ = load_one_annotation(os.path.join(bg_anno_dir, file_name + '.mat'))

        if flag_:
            record_file.write('Skipped %s for multi objects\n' % file_name)
            continue

        bg_img = os.path.join(bg_img_dir, file_name + '.JPEG')
        bg_mask = np.array(Image.open(os.path.join(bg_mask_dir, file_name + '.JPEG')))

        #if not bg_mask.shape[0] == bg_img.shape[0] and bg_mask.shape[1] == bg_img.shape[1]:
            #bg_mask = cv2.resize(mask, (bg_img.shape[1], bg_img.shape[0]), interpolation=cv2.INTER_NEAREST)

        # not sure why this is necessary, took from OccludedPascal3D+ code
        #bg_bbox = bbt.from_numpy(bg_bbox, image_boundary=img.shape[0:2], sorts=('x0', 'y0', 'x1', 'y1'))

        generate_composite_image(bg_img, bg_w, bg_h, bg_mask, bg_bbox)

In [14]:
categories = ['aeroplane', 'bicycle', 'bus', 'car', 'motorbike', 'train']

for cate in categories:
    print('Start cate: ', cate)
    tem = open('generating_record_%s_1030.txt' % cate, 'w')
    file_list_ = open(bg_list_path % cate).readlines()
    file_list_ = [tem.strip('\n') for tem in file_list_]
    bg_img_path_ = bg_img_path % cate
    bg_anno_path_ = bg_anno_path % cate
    bg_mask_path_ = bg_mask_path % cate

    generate_dataset(cate, file_list_, bg_img_path_, bg_anno_path_, bg_mask_path_, tem)

Start cate:  aeroplane
Start cate:  bicycle
Start cate:  bus
Start cate:  car
Start cate:  motorbike
Start cate:  train
